# 🏁 Add-on Module 10 – End-to-End Data Engineering Project

## Objective
Bring together everything learned across prior modules to design and implement a **real-world ETL (Extract-Transform-Load)** workflow using Pandas.

This project simulates a retail company consolidating sales data from multiple sources (SQL database, REST API, and CSV files) and delivering analytical insights.

**Core Concepts**  
→ Efficient extraction and merging from heterogeneous data sources  
→ Data cleaning, enrichment, and aggregation  
→ Performance optimization and storage back-end integration

## 1️⃣ Extract – Loading Data from Multiple Sources

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import requests, io

# Simulated SQL source
engine = create_engine('sqlite:///:memory:')
sales_sql = pd.DataFrame({
    'order_id': [1, 2, 3, 4],
    'product_id': [101, 102, 103, 104],
    'quantity': [2, 1, 4, 3],
    'price': [100, 250, 50, 300]
})
sales_sql.to_sql('sales', con=engine, index=False)
sales_db = pd.read_sql('SELECT * FROM sales', con=engine)

# Simulated REST API source
url = 'https://jsonplaceholder.typicode.com/users'
api_data = requests.get(url).json()
customers_api = pd.json_normalize(api_data)[['id', 'name', 'address.city']].rename(columns={'id':'customer_id'})

# Simulated CSV source (inventory)
csv_data = io.StringIO('product_id,stock\n101,20\n102,5\n103,50\n104,0')
inventory = pd.read_csv(csv_data)

sales_db.head(), customers_api.head(), inventory.head()

## 2️⃣ Transform – Data Cleaning & Feature Engineering

In [ ]:
# Compute total sale value
sales_db['total_value'] = sales_db['quantity'] * sales_db['price']

# Simulate adding a customer column (for join)
sales_db['customer_id'] = [1,2,3,4]

# Merge all sources
merged = sales_db.merge(customers_api, on='customer_id', how='left').merge(inventory, on='product_id', how='left')

# Handle missing stock
merged['stock'] = merged['stock'].fillna(0)

# Add new feature: stock status
merged['in_stock'] = merged['stock'] > 0
merged.head()

## 3️⃣ Enrich – Add External Information (e.g., Currency Rates)

In [ ]:
# Mock API for currency conversion
eur_rate = 0.93  # suppose we fetched this from exchangerate API
merged['total_value_eur'] = merged['total_value'] * eur_rate
merged[['order_id','name','total_value','total_value_eur']].head()

## 4️⃣ Analyze – Aggregation & Insights

In [ ]:
# Compute total sales by city
city_sales = merged.groupby('address.city')['total_value'].sum().reset_index().rename(columns={'address.city':'city'})

# Top products by revenue
product_perf = merged.groupby('product_id')['total_value'].sum().sort_values(ascending=False)

print(city_sales)  # summary by city
print('\nTop products by revenue:')
print(product_perf)

## 5️⃣ Load – Store Results to Database and Parquet

In [ ]:
# Write aggregates back to SQL
city_sales.to_sql('city_sales_summary', con=engine, index=False, if_exists='replace')

# Save for analytics
merged.to_parquet('sales_enriched.parquet', index=False)
print('✅ Data written to database and Parquet file successfully.')

## 6️⃣ Performance Optimization Tips

In [ ]:
# Vectorized operations
import numpy as np
merged['discount'] = np.where(merged['total_value'] > 400, 0.1, 0.05)

# Memory usage profiling
print('Memory usage (MB):', merged.memory_usage(deep=True).sum() / 1e6)

# Use categoricals for high-cardinality columns
merged['address.city'] = merged['address.city'].astype('category')

## 🧩 Real-World Problem 1 – Retail Sales Monitoring

**Scenario:** Monitor daily sales from multiple warehouses and update a dashboard in real time.

**Steps:**
1. Fetch sales data from API every hour.  
2. Store it in SQL via `to_sql()`.  
3. Run a Pandas job to aggregate and visualize daily KPIs (e.g., total sales, average order value, out-of-stock items).

## 🧩 Real-World Problem 2 – Customer 360° Profile Integration

**Scenario:** Merge data from CRM API, sales database, and marketing CSV to create a unified customer profile.

**Approach:**
- Join on customer IDs across sources.  
- Derive features like recency, frequency, monetary value (RFM).  
- Export profiles to Parquet or a warehouse for ML use.

## 🧠 Under the Hood
- **SQLAlchemy** manages database connections and transaction safety.  
- **Pandas I/O API** handles CSV/JSON/SQL adapters through modular backends.  
- **Parquet Engine (PyArrow)** enables columnar compression and zero-copy read paths.  
- **NumPy Vectorization** reduces Python loops in transform steps.

## ✅ Best Practices
- Validate schema consistency before merges.  
- Automate extract/transform/load jobs with Airflow or Prefect.  
- Profile pipelines with `cProfile` or `line_profiler`.  
- Use `.query()` and `.assign()` for clear, chainable transforms.  
- Compress large files using Snappy or ZSTD in Parquet.

## ⚡ Challenge Exercise

Design an automated ETL system that runs daily to:
1. Pull customer and transaction data from two APIs.  
2. Store raw data in a PostgreSQL staging area.  
3. Transform it using Pandas (handling missing values, date normalization, and currency conversion).  
4. Write clean data to an analytics database and a Parquet data lake.  
5. Generate daily KPIs and email them as an HTML report to stakeholders.